In [ ]:
#  Copyright (c) 2023 Snowflake Computing Inc. All rights reserved.

import streamlit as st
st.markdown("# 📈 Telco Churn Model")
st.write("Lets build a Telco Churn Model that will predict probability of churn for a telco company based on several user features.")
st.write("Note this notebook builds upon the [Snowflake Hex Quickstart](https://quickstarts.snowflake.com/guide/hex-churn-model/index.html). Hence minimal changes have been done to add other capabilities such as scikit-learn pipelines, hyper-parameter tuning, model deployment etc.""")

In [ ]:
st.markdown("## Import python packages")
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE 
# We can also use Snowpark for our analyses!
import altair as alt

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()
print(session)

In [ ]:
select * from SNOWPUBLIC.STREAMLIT.telco_churn_raw_data_DEMO;

In [ ]:
telco_churn_snow_df = cells.CELL19.to_df()
st.dataframe(telco_churn_snow_df)

In [ ]:
st.markdown("# Exploratory Data Analysis (EDA) \n Machine learning models thrive on clean and well-organized data. To ensure our models perform at their best, we'll investigate our dataset to address any missing values and visualize the distributions of each column.")

In [ ]:
st.markdown("## Basic Summary Statistics") 
st.dataframe(telco_churn_snow_df.describe())

In [ ]:
st.markdown("## Checking nulls with Pandas") 
st.write("As can be seen, there is no null value in any of the feature columns")
telco_churn_pdf = telco_churn_snow_df.to_pandas()
st.dataframe(telco_churn_pdf.isnull().sum())

In [ ]:
st.markdown("## Checking the distribution of the features.") 
columns = telco_churn_pdf.columns
num_columns_for_display = 2
col1, col2 = st.columns(num_columns_for_display)
index = 0
for col in columns:
    source = pd.DataFrame(telco_churn_pdf[col])
    chrt = alt.Chart(source).mark_bar().encode(
    alt.X(f"{col}:Q", bin=True),
    y='count()',
    )
    if index % num_columns_for_display == 0:
        with col1: 
            st.altair_chart(chrt)
    elif index % num_columns_for_display == 1:
        with col2: 
            st.altair_chart(chrt)
    index = index + 1

In [ ]:
st.markdown("## Big data processing with Snowpark Dataframes")
import time
start = time.time()
telco_churn_snow_df.group_by('"Churn"').count()
end = time.time()
st.markdown(f"Total Time with Snowpark: {end-start}")

start = time.time()
telco_churn_snow_pdf = telco_churn_snow_df.to_pandas()
end_mid = time.time()
telco_churn_snow_pdf.groupby("Churn").count()
end = time.time()
st.markdown(f"Total Time with Pandas: {end-start}")

st.markdown("Why is Pandas slower than Snowpark?")
st.markdown(f"I/O time to convert to Pandas dataframe: {end_mid-start}")
st.markdown(f"Processing time with Pandas dataframe: {end-end_mid}")
st.markdown(f"I/O account for {(end_mid-start)/(end-start)*100:.2f}% of processing time")

In [ ]:
st.markdown("## Understanding Churn Rate - Imbalanced dataset \n If you want to understand a model, you need to know its weaknesses. When the target variable has one class that is much more frequent than the other, your data is imbalanced. This causes issues when evaluating models since both classes don't get equal attention.\n In contrast to modeling an imbalanced dataset, a model trained on balanced data sees an equal amount of observations per class. By eliminating the imbalance, we also eliminate the model's potential to achieve high metric scores due to bias towards a majority class. This means that when we evaluate our model, the metrics can capture a better representation of how well the model does at making valuable predictions.")
st.dataframe(telco_churn_snow_df.group_by('"Churn"').count())

In [ ]:
st.markdown("# Feature Engineering - include balancing the dataset and standard scaling \n To prepare our data for our model, we'll need to handle the imbalanced data problem by upsampling our dataset. Once that's taken care of, we'll use scikit-learn to preprocess our data into a format that the model expects. This means scaling our features and splitting our data into training and testing datasets.")
# Extract the training features
features_names = [col for col in telco_churn_pdf.columns if col not in ['Churn']]
features = telco_churn_pdf[features_names]

# extract the target
target = telco_churn_pdf['Churn']
st.markdown("## Lets balance the dataset.")
# upsample the minority class in the dataset
upsampler = SMOTE(random_state = 111)
features, target = upsampler.fit_resample(features, target)
st.dataframe(features.head())

st.markdown("## Upsampled data.")
upsampled_data = pd.concat([features, target], axis=1)
upsampled_data.reset_index(inplace=True)
upsampled_data.rename(columns={'index': 'INDEX'}, inplace=True)
st.dataframe(upsampled_data.head())

In [ ]:
upsampled_data = session.create_dataframe(upsampled_data)
# Get the list of column names from the dataset
feature_names_input = [c for c in upsampled_data.columns if c != '"Churn"' and c != "INDEX"]

In [ ]:
st.dataframe(upsampled_data[feature_names_input].to_pandas())

We can perform StandardScaler preprocessing via sklearn to process in-memory or Snowpark ML preprocessing for pushdown compute. Note the similarity between the APIs used and output results.

In [ ]:
st.markdown("## Sci-kit learn Preprocessing with Pandas DataFrames")
import sklearn.preprocessing as pp_original
# Initialize a StandardScaler object with input and output column names
scaler = pp_original.StandardScaler()
features_pdf = upsampled_data[feature_names_input].to_pandas()
# Fit the scaler to the dataset
scaler.fit(features_pdf)

# Transform the dataset using the fitted scaler
scaled_features = scaler.transform(features_pdf)
st.dataframe(scaled_features)

In [ ]:
st.markdown("## Snowpark ML preprocessing with Snowpark DataFrames")
import snowflake.ml.modeling.preprocessing as pp
# Get the list of column names from the dataset
feature_names_input = [c for c in upsampled_data.columns if c != '"Churn"' and c != "INDEX"]


# Initialize a StandardScaler object with input and output column names
scaler = pp.StandardScaler(
    input_cols=feature_names_input,
    output_cols=feature_names_input
)

# Fit the scaler to the dataset
scaler.fit(upsampled_data)

# Transform the dataset using the fitted scaler
scaled_features = scaler.transform(upsampled_data)
st.dataframe(scaled_features)

In [ ]:
st.markdown("## Let's perform the train test split using 80/20.")

# Define the target variable (label) column name
label = ['"Churn"']

# Define the output column name for the predicted label
output_label = ['"predicted_churn"']

# Split the scaled_features dataset into training and testing sets with an 80/20 ratio
training, testing = scaled_features.random_split(weights=[0.8, 0.2], seed=111)

In [ ]:
from snowflake.ml.modeling.ensemble import RandomForestClassifier
# Training
st.markdown("# Model Training - Random Forest Classifier \n The mystery model of the day is a [random forest classifier](https://towardsdatascience.com/understanding-random-forest-58381e0602d2). I'll spare you the details on how it works, but in short, it creates an ensemble of smaller models that all make predictions on the same data. Whichever prediction has the most votes is the final prediction that the model goes with.")
# Initialize a RandomForestClassifier object with input, label, and output column names
model = RandomForestClassifier(
    input_cols=feature_names_input,
    label_cols=label,
    output_cols=output_label,
)

In [ ]:
# Train the RandomForestClassifier model using the training set
model.fit(training)

In [ ]:
# Predict the target variable (churn) for the testing set using the trained model
results = model.predict(testing)

In [ ]:
testing

In [ ]:
st.markdown("""# Model Evaluation

Model evaluation is all about checking how well our machine learning model is doing by comparing its predictions to the actual outcomes. In this case, we're going to focus on *recall* instead of *accuracy* as our main evaluation metric. """)

# return only the predicted churn values
predictions = results.to_pandas().sort_values("INDEX")[output_label].astype(int).to_numpy().flatten()
actual = testing.to_pandas().sort_values("INDEX")[['Churn']].to_numpy().flatten()


In [ ]:


st.markdown("""# Feature Importance

Feature importance is all about figuring out which input variables are the real MVPs when it comes to making predictions with our machine learning model. We'll find out which features are the most important by looking at how much they contribute to the model's overall performance.""")
rf = model.to_sklearn()
importances = pd.DataFrame(
    list(zip(features.columns, rf.feature_importances_)),
    columns=["feature", "importance"],
)

bar_chart = alt.Chart(importances).mark_bar().encode(
    x="importance:Q",
    y=alt.Y("feature:N", sort="-x")
)
st.altair_chart(bar_chart, use_container_width=True)

In [ ]:
st.markdown("""# Predicting churn for a new user
Using our trained random forest model, we can make predictions that tell us whether a new customer will churn or not.""")

account_weeks = "10"
data_usage = "1.7"
mins_per_month = "82"
daytime_calls = "67"
customer_service_calls = "4"
monthly_charge = "37"
roam_mins = "0"
overage_fee = "9.5"
renewed_contract = "true"
has_data_plan = "true"
user_vector = np.array([
    account_weeks,
    1 if renewed_contract else 0,
    1 if has_data_plan else 0,
    data_usage,
    customer_service_calls,
    mins_per_month,
    daytime_calls,
    monthly_charge,
    overage_fee,
    roam_mins,
]).reshape(1,-1)

user_dataframe = pd.DataFrame(user_vector, columns=[f'"{_}"' for _ in features.columns])


In [ ]:
st.markdown("#### Input dataframe for new user")
st.dataframe(user_dataframe)
user_vector = scaler.transform(user_dataframe)

In [ ]:
st.dataframe(model.predict(user_vector)[['"predicted_churn"']].values)

In [ ]:
st.markdown("#### Scaled dataframe for new user")
st.dataframe(user_vector)
st.markdown("#### Prediction")
predicted_value = model.predict(user_vector)[['"predicted_churn"']].values.astype(int).flatten()
user_probability = model.predict_proba(user_vector)
probability_of_prediction = max(user_probability[user_probability.columns[-2:]].values[0]) * 100
prediction = 'churn' if predicted_value == 1 else 'not churn'
print(prediction)

In [ ]:
st.markdown("""# Predicting churn for a new user
Using our trained random forest model, we can make predictions that tell us whether a new customer will churn or not.""")

col1, col2 = st.columns(2)
with col1: 
    account_weeks = st.slider("AccountWeeks", int(features["AccountWeeks"].min()) , int(features["AccountWeeks"].max()))
    data_usage = st.slider("DataUsage", int(features["DataUsage"].min()) , int(features["DataUsage"].max()))
    mins_per_month = st.slider("DayMins", int(features["DayMins"].min()) , int(features["DayMins"].max()))
    daytime_calls = st.slider("DayCalls", int(features["DayCalls"].min()) , int(features["DayCalls"].max()))
    renewed_contract =  st.selectbox("Renewed Contract?",('true','false'))
with col2: 
    monthly_charge = st.slider("MonthlyCharge", int(features["MonthlyCharge"].min()) , int(features["MonthlyCharge"].max()))
    roam_mins = st.slider("RoamMins", int(features["RoamMins"].min()) , int(features["RoamMins"].max()))
    customer_service_calls = st.slider("CustServCalls", int(features["CustServCalls"].min()) , int(features["CustServCalls"].max()))
    overage_fee = st.slider("OverageFee", int(features["OverageFee"].min()) , int(features["OverageFee"].max()))
    has_data_plan = st.selectbox("Has Data Plan?",('true','false'))
user_vector = np.array([
    account_weeks,
    1 if renewed_contract else 0,
    1 if has_data_plan else 0,
    data_usage,
    customer_service_calls,
    mins_per_month,
    daytime_calls,
    monthly_charge,
    overage_fee,
    roam_mins,
]).reshape(1,-1)

user_dataframe = pd.DataFrame(user_vector, columns=[f'"{_}"' for _ in features.columns])
st.markdown("#### Input dataframe for new user")
st.dataframe(user_dataframe)
user_vector = scaler.transform(user_dataframe)
st.markdown("#### Scaled dataframe for new user")
st.dataframe(user_vector)
st.markdown("#### Prediction")
predicted_value = model.predict(user_vector)[['"predicted_churn"']].values.astype(int).flatten()
user_probability = model.predict_proba(user_vector)
probability_of_prediction = max(user_probability[user_probability.columns[-2:]].values[0]) * 100
prediction = 'churn' if predicted_value == 1 else 'not churn'
print(prediction)


In [ ]:
st.markdown("Exporting Model with Timestamp")
import pickle
import datetime
filename = f'telco-eda-model-{datetime.datetime.now()}.pkl'

pickle.dump(model, open(filename,'wb'))
print(f"Saved to {filename}")